Primeros pasos que se dió para comprender los datos y lo que se podía hacer

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import os
import constants as c
import time
import utils

from alpha_vantage.techindicators import TechIndicators
from datetime import datetime, timedelta

//anaconda3/envs/bsm03/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
COLUMNS = ['symbol', 'date', 'close', 'volume', 'open', 'high', 'low']
COLUMNS_TECH = ['symbol', 'date', 'MACD_Signal', 'MACD_Hist', 'MACD', 'SlowK', 'SlowD',
       'Chaikin A/D', 'OBV', 'EMA21', 'SMA21', 'WMA21', 'RSI21', 'ADX21',
       'CCI21', 'Aroon Up21', 'Aroon Down21', 'Real Lower Band21',
       'Real Upper Band21', 'Real Middle Band21', 'EMA28', 'SMA28', 'WMA28',
       'RSI28', 'ADX28', 'CCI28', 'Aroon Down28', 'Aroon Up28',
       'Real Lower Band28', 'Real Upper Band28', 'Real Middle Band28', 'EMA50',
       'SMA50', 'WMA50', 'RSI50', 'ADX50', 'CCI50', 'Aroon Up50',
       'Aroon Down50', 'Real Middle Band50', 'Real Lower Band50',
       'Real Upper Band50']

In [3]:
symbol = "AAPL"

In [6]:
p = f'./web_app/data/close_{symbol}.csv'
t = f'./web_app/data/tech_{symbol}.csv'

In [7]:
data = pd.read_csv(p, names=COLUMNS)
data['date'] = pd.to_datetime(data['date'])

In [9]:
data['dif1'] = data['close'].pct_change(1)

In [10]:
data.head()

,symbol,date,close,volume,open,high,low,dif1
0,AAPL,2006-01-03,74.75,28829800,72.332,74.75,72.25,NaN
1,AAPL,2006-01-04,74.97,22128700,75.130,75.98,74.50,0.002943
2,AAPL,2006-01-05,74.38,16050800,74.830,74.90,73.75,-0.007870
3,AAPL,2006-01-06,76.30,25159200,75.240,76.70,74.55,0.025813
4,AAPL,2006-01-09,76.05,24108600,76.730,77.20,75.74,-0.003277


In [17]:
data_sorted = data.sort_values(by=['date'], ascending=False)

In [18]:
data_sorted['diff2'] = data_sorted['close'].pct_change(1)

In [19]:
data_sorted.head()

,symbol,date,close,volume,open,high,low,dif1,diff2
3631,AAPL,2020-06-08,333.46,23851975,330.250,333.60,327.32,0.005913,NaN
3630,AAPL,2020-06-05,331.50,34312550,323.350,331.75,323.23,0.028481,-0.005878
3629,AAPL,2020-06-04,322.32,21890091,324.390,325.62,320.78,-0.008612,-0.027692
3628,AAPL,2020-06-03,325.12,26122804,324.660,326.20,322.30,0.005505,0.008687
3627,AAPL,2020-06-02,323.34,21754815,320.745,323.44,318.93,0.004629,-0.005475


In [23]:
331.50 + 331.50 * 0.005878

333.448557

In [21]:
(331.50 - 333.46) / 333.46

-0.005877766448749414

In [18]:
data_t = pd.read_csv(t, names=COLUMNS_TECH)
data_t['date'] = pd.to_datetime(data_t['date'])

In [19]:
df = pd.merge(data, data_t, on=['symbol', 'date'], how='inner')

In [23]:
df.head()

,symbol,date,close,volume,open,high,low,MACD_Signal,MACD_Hist,MACD,...,SMA50,WMA50,RSI50,ADX50,CCI50,Aroon Up50,Aroon Down50,Real Middle Band50,Real Lower Band50,Real Upper Band50
0,AAPL,2006-01-03,74.75,28829800,72.332,74.75,72.25,2.2043,-0.4653,1.7390,...,66.7126,70.2124,63.9946,23.5817,81.1462,74.0,12.0,66.7126,53.5292,79.8960
1,AAPL,2006-01-04,74.97,22128700,75.130,75.98,74.50,2.1179,-0.3457,1.7722,...,67.0988,70.5362,64.1541,23.6276,91.9853,100.0,10.0,67.0988,54.1032,80.0944
2,AAPL,2006-01-05,74.38,16050800,74.830,74.90,73.75,2.0405,-0.3095,1.7311,...,67.4506,70.8218,63.3856,23.6319,80.4309,98.0,8.0,67.4506,54.6392,80.2620
3,AAPL,2006-01-06,76.30,25159200,75.240,76.70,74.55,1.9989,-0.1667,1.8322,...,67.8546,71.1688,64.7863,23.6942,95.2311,100.0,6.0,67.8546,55.2278,80.4814
4,AAPL,2006-01-09,76.05,24108600,76.730,77.20,75.74,1.9732,-0.1026,1.8706,...,68.2350,71.4902,64.4587,23.7709,98.4107,100.0,4.0,68.2350,55.7909,80.6792


In [29]:
cols = list(df.columns)
cols.remove('date')
cols.remove('symbol')
pd.melt(df, id_vars=['date'], value_vars=cols)

,date,variable,value
0,2006-01-03,close,74.7500
1,2006-01-04,close,74.9700
2,2006-01-05,close,74.3800
3,2006-01-06,close,76.3000
4,2006-01-09,close,76.0500
...,...,...,...
163255,2020-05-27,Real Upper Band50,333.7417
163256,2020-05-28,Real Upper Band50,335.5433
163257,2020-05-29,Real Upper Band50,337.0210
163258,2020-06-01,Real Upper Band50,338.5654


In [3]:
data_daily = utils.get_daily(symbol)

In [4]:
data_daily.head()

,date,open,high,low,close,volume,symbol
0,2006-01-03,26.25,27.0010,26.10,26.84,79973000,MSFT
1,2006-01-04,26.77,27.0801,26.77,26.97,57975600,MSFT
2,2006-01-05,26.93,27.1300,26.91,26.99,48245500,MSFT
3,2006-01-06,26.88,27.0000,26.49,26.91,100963000,MSFT
4,2006-01-09,26.93,27.0700,26.76,26.86,55625000,MSFT


In [8]:
# data_tech = get_technical(symbol)

In [7]:
pd.read_csv('data/close_MSFT.csv', names=[1, 2, 3, 4, 5, 6, 7]).dtypes

1     object
2     object
3    float64
4      int64
5    float64
6    float64
7    float64
dtype: object